In [1]:
import sys
import os
from pathlib import Path
sys.path.append(os.getcwd())
sys.path.append(str(Path(os.getcwd()).parent))
sys.path.append(os.path.join(str(Path(os.getcwd()).parent), 'NEAT_LNC_node'))
import torch
import NEAT_LNC_node as neat
import pprint
from utils import *
from configuration import *
from NEAT_args import *

if not os.path.exists('./evaluation/'):
    os.makedirs('./evaluation/')
    
args = parser.parse_args([])
EA_args = EA_parser.parse_args([])

In [2]:
args = parser.parse_args([])
results = torch.zeros([10,13,3,3,4])

for ds in range(13):
    args.DEVICE = 'cpu'
    args.DATASET = ds
    args = FormulateArgs(args)

    valid_loader, datainfo = GetDataLoader(args, 'valid', path='../dataset/')
    test_loader , datainfo = GetDataLoader(args, 'test',  path='../dataset/')
    pprint.pprint(datainfo)

    EA_args.num_inputs = datainfo['N_feature']
    EA_args.num_outputs = datainfo['N_class']

    for seed in range(10):
        for idxtr, eps_train in enumerate([0., 0.05, 0.1]):
            for idxte, eps_test in enumerate([0., 0.05, 0.1]):
                
                args.SEED = seed
                args.e_train = eps_test

                
                modelname = f"NEAT_data_{datainfo['dataname']}_seed_{args.SEED:02d}_epsilon_{eps_train}.model"
                modelpath = f'./models/{modelname}'
                if os.path.exists(modelpath):
                    model = torch.load(modelpath)

                    config = neat.Config(neat.PNCGenome, neat.DefaultReproduction, neat.DefaultSpeciesSet, neat.DefaultStagnation, EA_args)
                    neat.eval_genome(model, config, valid_loader, args)
                    accv = model.accuracy
                    stdv = model.std

                    neat.eval_genome(model, config, test_loader, args)
                    acct = model.accuracy
                    stdt = model.std
                    
                else:
                    accv = 0.
                    stdv = 0.
                    acct = 0.
                    stdt = 0.
                    
    
                results[seed,ds,idxtr,idxte,0] = torch.tensor(accv)
                results[seed,ds,idxtr,idxte,1] = torch.tensor(stdv)

                results[seed,ds,idxtr,idxte,2] = torch.tensor(acct)
                results[seed,ds,idxtr,idxte,3] = torch.tensor(stdt)

                dsname = datainfo['dataname']
                print(f'Processing {dsname:<10}, seed {seed:>3}, handling epsilon {eps_train:>6}, testing epsilon {eps_test:>6}, valid acc {accv:.4f}, valid std {stdv:>.4f}, test acc {acct:>.4f}, test std {stdt:>.4f}')
    torch.save(results[:,ds,:,:,:], f"./evaluation/result_data_{datainfo['dataname']}.matrix")
    
torch.save(results.permute(1,0,2,3,4), f"result.matrix")

{'N_class': 2,
 'N_feature': 6,
 'N_test': 25,
 'N_train': 70,
 'N_valid': 23,
 'dataname': 'acuteinflammation'}
Processing acuteinflammation, seed   0, handling epsilon    0.0, testing epsilon    0.0, valid acc 0.6087, valid std 0.0000, test acc 0.3200, test std 0.0000
Processing acuteinflammation, seed   0, handling epsilon    0.0, testing epsilon   0.05, valid acc 0.6174, valid std 0.0379, test acc 0.3320, test std 0.0523
Processing acuteinflammation, seed   0, handling epsilon    0.0, testing epsilon    0.1, valid acc 0.6283, valid std 0.0853, test acc 0.4360, test std 0.2105
Processing acuteinflammation, seed   0, handling epsilon   0.05, testing epsilon    0.0, valid acc 1.0000, valid std 0.0000, test acc 1.0000, test std 0.0000
Processing acuteinflammation, seed   0, handling epsilon   0.05, testing epsilon   0.05, valid acc 0.9913, valid std 0.0261, test acc 0.9980, test std 0.0087
Processing acuteinflammation, seed   0, handling epsilon   0.05, testing epsilon    0.1, valid ac